## Analysis of cell centroid displacements

1. caclulate total cell displacement 2. average displacement for each tissue was measured by dividing the total cell displacement by the number of cells in the tissue, which was performed in excel

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
### step 1: NET DISPLACEMENTS CALCULATE

# Define the folder containing your input CSV files
input_folder = "/Users/sierraschwabach/Documents/All_tracks/new_rot/"  # Replace with your folder path

# Function to calculate net displacement
def calculate_net_displacement(input_folder):
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
    
    for file in csv_files:
        # Read the CSV file
        file_path = os.path.join(input_folder, file)
        data = pd.read_csv(file_path)
        
        # Group by cell to calculate starting and ending positions
        net_displacement = (
            data.groupby('cell')
            .agg(
                start_row=('centroid_row', 'first'),
                start_col=('centroid_col', 'first'),
                end_row=('centroid_row', 'last'),
                end_col=('centroid_col', 'last')
            )
        )
        
        # Calculate net displacement (Euclidean distance)
        net_displacement['net_displacement'] = np.sqrt(
            (net_displacement['end_row'] - net_displacement['start_row'])**2 +
            (net_displacement['end_col'] - net_displacement['start_col'])**2
        )
        
        # Save the results to a new CSV file
        output_file = os.path.join(input_folder, file.replace('.csv', '_net_displacement.csv'))
        net_displacement.to_csv(output_file)
        print(f"Net displacement saved to {output_file}")

# Run the function
calculate_net_displacement(input_folder)

print(f"Net displacement saved to {output_file}")


In [ ]:
#### step2: AGGREGATE

# Folder containing displacement CSVs
input_folder = "/Users/sierraschwabach/Documents/displacements_microns/"  # Replace with your folder path

# Function to aggregate displacements with condition labels and file names
def aggregate_displacements_with_filenames(input_folder):
    displacement_data = []
    
    # Find all displacement CSVs
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('_displacement.csv')]
    
    for file in csv_files:
        # Read the CSV
        file_path = os.path.join(input_folder, file)
        data = pd.read_csv(file_path)
        
        # Determine condition from file name
        if "10930_apical" in file:
            condition = "apical"
        elif "n1032_" in file:
            condition = "n1032"
        elif "rot_apical" in file:
            condition = "rot_apical"
        elif "rot_basal" in file:
            condition = "rot_basal"
        else:
            condition = "10930_basal"
        
        # Add condition and file name columns
        data['condition'] = condition
        data['source_file'] = file  # Add the original file name
        displacement_data.append(data)
    
    # Combine all data
    return pd.concat(displacement_data, ignore_index=True)

# Aggregate data
displacement_df = aggregate_displacements_with_filenames(input_folder)

# Save the aggregated data
output_file = os.path.join(input_folder, "aggregated_displacements_with_filenames.csv")
displacement_df.to_csv(output_file, index=False)
print(f"Aggregated displacements with file names saved to {output_file}")



In [ ]:
## measure total displacement

# Folder containing displacement CSVs
input_folder = "/Users/sierraschwabach/Documents/displacements_microns/"  # Replace with your folder path


# Function to calculate displacement metrics
def calculate_displacement_metrics_with_conditions(input_folder):
    metrics = []
    
    # Find all displacement CSVs
    csv_files = [f for f in os.listdir(input_folder) if f.endswith('_displacement.csv')]
    
    for file in csv_files:
        # Read the CSV
        file_path = os.path.join(input_folder, file)
        data = pd.read_csv(file_path)
        
        # Determine condition from file name
        if "10930_apical" in file:
            condition = "apical"
        elif "n1032_" in file:
            condition = "n1032"
        elif "rot_apical" in file:
            condition = "rot_apical"
        elif "rot_basal" in file:
            condition = "rot_basal"
        else:
            condition = "10930_basal"
        
        # Calculate metrics
        total_displacement = data['net_dis_um'].sum()
        median_displacement = data['net_dis_um'].median()
        num_cells = data['cell'].nunique()
        
        # Store metrics
        metrics.append({
            'sample': file.replace('_displacement.csv', ''),
            'condition': condition,
            'total_displacement': total_displacement,
            'median_displacement': median_displacement,
            'num_cells': num_cells
        })
    
    # Convert to DataFrame
    return pd.DataFrame(metrics)


# Calculate metrics
metrics_df = calculate_displacement_metrics_with_conditions(input_folder)

# Save metrics to CSV
output_file = os.path.join(input_folder, "displacement_metrics_with_conditions.csv")
metrics_df.to_csv(output_file, index=False)
print(f"Displacement metrics with conditions saved to {output_file}")
